In [1]:
pip install scx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 33.9 MB/s eta 0:00:00


In [2]:
from scx.optimize import Model

In [29]:
transport = [
    {
        'origin':'P1',
        'destination':'IS_A',
        'cost_per_pellet':13.58
    },
    {
        'origin':'P2',
        'destination':'IS_A',
        'cost_per_pellet':16.54
    },
    {
        'origin':'IS_A',
        'destination':'S_X',
        'cost_per_pellet':7.57
    },
    {
        'origin':'IS_A',
        'destination':'S_Y',
        'cost_per_pellet':16.46
    },
    {
        'origin':'IS_B',
        'destination':'S_X',
        'cost_per_pellet':9.45
    },
    {
        'origin':'IS_B',
        'destination':'S_Y',
        'cost_per_pellet':15.64
    },
    {
        'origin':'P1',
        'destination':'IS_B',
        'cost_per_pellet':12.6
    },
    {
        'origin':'P2',
        'destination':'IS_B',
        'cost_per_pellet':16.2
    },
    {
        'origin':'P3',
        'destination':'IS_A',
        'cost_per_pellet':16.37
    },
    {
        'origin':'P3',
        'destination':'IS_B',
        'cost_per_pellet':10.81
    },
    {
        'origin':'IS_A',
        'destination':'S_Z',
        'cost_per_pellet':15.69
    },
    {
        'origin':'IS_B',
        'destination':'S_Z',
        'cost_per_pellet':15.41
    }
]

supply = [
    {
        'name':'P1',
        'supply':280
    },
    {
        'name':'P2',
        'supply':360
    },
    {
        'name':'P3',
        'supply':340
    }

]

demand = [
    {
        'name':'S_X',
        'demand':315
    },
    {
        'name':'S_Y',
        'demand':335
    },
    {
        'name':'S_Z',
        'demand':330
    }
]

Inspection_site = [
    {
        'name':'IS_A'
    },
    {
        'name':'IS_B'
    }
]

In [30]:
for t in transport:
  t['amt'] = Model.variable(name=f"{t['origin']}_{t['destination']}__amt",lowBound=0)
  t['cost'] = t['cost_per_pellet']*0.9687

In [31]:
print(transport[-1])

{'origin': 'IS_B', 'destination': 'S_Z', 'cost_per_pellet': 15.41, 'amt': IS_B_S_Z__amt, 'cost': 14.927667}


In [32]:
#Initialize the model
my_model = Model(name = 'GCC',sense = 'minimize')

#Add the objective function
my_model.add_objective(
    fn = Model.sum([t['cost']*t['amt'] for t in transport])
)

#Add the constraint
##Demand
for d in demand:
  my_model.add_constraint(name=f"{d['name']}__demand",
                          fn = Model.sum([t['amt'] for t in transport if t['destination'] == d['name']]) >= d['demand'])

##Supply
for s in supply:
  my_model.add_constraint(name = f"{s['name']}__supply",
                          fn = Model.sum([t['amt'] for t in transport if t['origin'] == s['name']]) <= s['supply'])

##Conservation of flow
for i in Inspection_site:
  my_model.add_constraint(name = f"{i['name']}__cof",
                          fn = Model.sum([t['amt'] for t in transport if t['destination'] == i['name']]) == Model.sum([t['amt'] for t in transport if t['origin'] == i['name']]))

#Solve the model
my_model.solve()

In [33]:
my_model.show_formulation()

GCC:
MINIMIZE
7.333059*IS_A_S_X__amt + 15.944802000000001*IS_A_S_Y__amt + 15.198903*IS_A_S_Z__amt + 9.154214999999999*IS_B_S_X__amt + 15.150468*IS_B_S_Y__amt + 14.927667*IS_B_S_Z__amt + 13.154946*P1_IS_A__amt + 12.20562*P1_IS_B__amt + 16.022298*P2_IS_A__amt + 15.69294*P2_IS_B__amt + 15.857619000000001*P3_IS_A__amt + 10.471647*P3_IS_B__amt + 0.0
SUBJECT TO
S_X__demand: IS_A_S_X__amt + IS_B_S_X__amt >= 315

S_Y__demand: IS_A_S_Y__amt + IS_B_S_Y__amt >= 335

S_Z__demand: IS_A_S_Z__amt + IS_B_S_Z__amt >= 330

P1__supply: P1_IS_A__amt + P1_IS_B__amt <= 280

P2__supply: P2_IS_A__amt + P2_IS_B__amt <= 360

P3__supply: P3_IS_A__amt + P3_IS_B__amt <= 340

IS_A__cof: - IS_A_S_X__amt - IS_A_S_Y__amt - IS_A_S_Z__amt + P1_IS_A__amt
 + P2_IS_A__amt + P3_IS_A__amt = 0

IS_B__cof: - IS_B_S_X__amt - IS_B_S_Y__amt - IS_B_S_Z__amt + P1_IS_B__amt
 + P2_IS_B__amt + P3_IS_B__amt = 0

VARIABLES
IS_A_S_X__amt Continuous
IS_A_S_Y__amt Continuous
IS_A_S_Z__amt Continuous
IS_B_S_X__amt Continuous
IS_B_S_Y__amt C

In [34]:
my_model.show_outputs()

{'objective': 25042.590225,
 'status': 'Optimal',
 'variables': {'IS_A_S_X__amt': 315.0,
               'IS_A_S_Y__amt': 0.0,
               'IS_A_S_Z__amt': 0.0,
               'IS_B_S_X__amt': 0.0,
               'IS_B_S_Y__amt': 335.0,
               'IS_B_S_Z__amt': 330.0,
               'P1_IS_A__amt': 0.0,
               'P1_IS_B__amt': 280.0,
               'P2_IS_A__amt': 315.0,
               'P2_IS_B__amt': 45.0,
               'P3_IS_A__amt': 0.0,
               'P3_IS_B__amt': 340.0}}
